first we have to import keras,numpy library

In [7]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np


latent_dim measn no of features in wordembedding

In [11]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = '/content/fra.txt'

input,target text means original english and hindi words

input,target character means only chracters. 93 in french and   in english

In [12]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

open finction assign file pointer to f so we can acess file through f and then we use split function

In [14]:
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

In [16]:
type(lines)

list

In [24]:
lines[1]

'Hi.\tSalut !\tCC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #509819 (Aiji)'

output of lines is like this

'I saw you.\tJe vous ai vus.\tCC-BY 2.0 (France) Attribution: tatoeba.org #2247402 (CK) & #2527560 (sacredceltic)',

so we have to seprate both english and french word and also remove unwanted part as well.

**for the targets, and "\n" as "end sequence" character.**

In [19]:
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [20]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [21]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 92
Max sequence length for inputs: 16
Max sequence length for outputs: 59


In [22]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

In [ ]:
input_token_index

we have to give 3 input to the encoder-decoder so these three are inputs

it should be 3d array of size ( total words in file * max word lenghth * encoder representation ) 

In [25]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [26]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

encoder part here we just want state_h and state_c for decoder we ignore encoder output

we are using return_state = true so we get state of lstm also

In [27]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

In [28]:
encoder_states = [state_h, state_c]

In [29]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [30]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
#https://www.tensorflow.org/api_docs/python/tf/keras/Model

In [31]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 57s 7ms/step - loss: 1.1989 - accuracy: 0.7231 - val_loss: 1.0634 - val_accuracy: 0.7075
Epoch 2/100
8000/8000 [==============================] - 57s 7ms/step - loss: 0.8577 - accuracy: 0.7685 - val_loss: 0.8439 - val_accuracy: 0.7661
Epoch 3/100
8000/8000 [==============================] - 56s 7ms/step - loss: 0.6867 - accuracy: 0.8069 - val_loss: 0.7400 - val_accuracy: 0.7891
Epoch 4/100
8000/8000 [==============================] - 56s 7ms/step - loss: 0.5960 - accuracy: 0.8270 - val_loss: 0.6557 - val_accuracy: 0.8098
Epoch 5/100
8000/8000 [==============================] - 56s 7ms/step - loss: 0.5445 - accuracy: 0.8408 - val_loss: 0.6089 - val_accuracy: 0.8209
Epoch 6/100
7936/8000 [============================>.] - ETA: 0s - loss: 0.5066 - accuracy: 0.8515

KeyboardInterrupt: ignored

In [ ]:
model.save('s2s.h5')